<a href="https://colab.research.google.com/github/pei0217/fin_hw9_week13/blob/main/fin_hw9_week13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import random
import yfinance as yf


stock_symbols = ["AAPL", "NVDA", "2317.TW", "AMZN", "TSLA"]

# 計算回報率和風險（標準差）
def fetch_stock_data(symbols, period="2y"):
    stock_returns = []
    stock_risks = []

    for symbol in symbols:
        data = yf.download(symbol, period=period)
        # 計算每日回報率
        data["Return"] = data["Adj Close"].pct_change()
        avg_return = data["Return"].mean() * 252  # 年化回報率
        risk = data["Return"].std() * np.sqrt(252)  # 年化風險
        stock_returns.append(avg_return)
        stock_risks.append(risk)

    return stock_returns, stock_risks

# 爬取股票數據
stock_returns, stock_risks = fetch_stock_data(stock_symbols)

# 初始化參數
population_size = 20
num_generations = 100
mutation_rate = 0.1
budget = 3  # 最大選股數量限制

# 適應函數
def fitness(chromosome):
    total_return = np.dot(chromosome, stock_returns)
    total_risk = np.dot(chromosome, stock_risks)
    if sum(chromosome) > budget:  # 超出選股數量限制的個體
        return 1e-6  # 返回一個非常小的正值
    return total_return / (total_risk + 1e-6)  # 加小值避免除零

# 初始化族群
def initialize_population(size, n_stocks):
    return [np.random.randint(2, size=n_stocks) for _ in range(size)]

# 選擇：輪盤賭選擇法
def select(population, fitness_scores):
    fitness_scores = [max(score, 1e-6) for score in fitness_scores]  # 確保適應值為正
    fitness_sum = sum(fitness_scores)
    probabilities = [score / fitness_sum for score in fitness_scores]
    selected_index = np.random.choice(len(population), p=probabilities)
    return population[selected_index]

# 交配：單點交叉
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
    return child

# 突變：隨機翻轉一個基因
def mutate(chromosome):
    if random.random() < mutation_rate:
        mutation_point = np.random.randint(len(chromosome))
        chromosome[mutation_point] = 1 - chromosome[mutation_point]
    return chromosome

# 主遺傳算法流程
def genetic_algorithm():
    population = initialize_population(population_size, len(stock_returns))

    for generation in range(num_generations):
        fitness_scores = [fitness(individual) for individual in population]

        # 保留適應度最高的個體
        new_population = [population[np.argmax(fitness_scores)]]

        # 生成下一代
        while len(new_population) < population_size:
            parent1 = select(population, fitness_scores)
            parent2 = select(population, fitness_scores)
            child = crossover(parent1, parent2)
            child = mutate(child)
            new_population.append(child)

        population = new_population

    # 找到最優解
    fitness_scores = [fitness(individual) for individual in population]
    best_index = np.argmax(fitness_scores)
    best_chromosome = population[best_index]
    return best_chromosome, fitness(best_chromosome)

# 執行算法
best_solution, best_fitness = genetic_algorithm()
selected_stocks = [stock_symbols[i] for i, gene in enumerate(best_solution) if gene == 1]
print("最佳選股組合:", selected_stocks)
print("最佳適應值（回報率/風險）:", best_fitness)
print("總回報率:", sum(np.array(stock_returns)[best_solution]))
print("總風險:", sum(np.array(stock_risks)[best_solution]))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


最佳選股組合: ['NVDA']
最佳適應值（回報率/風險）: 2.3903484373279342
總回報率: 2.435185640776829
總風險: 1.3945279557804486
